In [18]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=60,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Maverick, I'm in Topgun", "Wow, that is so cool!")
get_history()

{'history': [HumanMessage(content="Hi I'm Maverick, I'm in Topgun"),
  AIMessage(content='Wow, that is so cool!')]}

In [19]:
add_message("Do you have anything to ask?", "yes, how's Iceman doing?")

get_history()

{'history': [HumanMessage(content="Hi I'm Maverick, I'm in Topgun"),
  AIMessage(content='Wow, that is so cool!'),
  HumanMessage(content='Do you have anything to ask?'),
  AIMessage(content="yes, how's Iceman doing?")]}

In [20]:
add_message(
    "I'm not sure,we fought yesterday again, yesterday.",
    "I thought that we are getting better, what happened?",
)

get_history()

{'history': [SystemMessage(content="The human introduces themselves as Maverick from Topgun. The AI expresses admiration for Maverick's role. Maverick asks if the AI has any questions."),
  AIMessage(content="yes, how's Iceman doing?"),
  HumanMessage(content="I'm not sure,we fought yesterday again, yesterday."),
  AIMessage(content='I thought that we are getting better, what happened?')]}

In [22]:
add_message(
    "Same as usual, I thought that we became a friend but...",
    "Look, Iceman is comming!",
)

get_history()

{'history': [SystemMessage(content="The human introduces themselves as Maverick from Topgun. The AI expresses admiration for Maverick's role and asks about Iceman. Maverick mentions they fought with Iceman yesterday because they thought they had become friends but things went back to usual."),
  AIMessage(content='Look, Iceman is comming!'),
  HumanMessage(content='Same as usual, I thought that we became a friend but...'),
  AIMessage(content='Look, Iceman is comming!')]}

In [23]:
add_message(
    "Oh, shit. What should I do.",
    "Say something!",
)

get_history()

{'history': [SystemMessage(content="The human introduces themselves as Maverick from Topgun. The AI expresses admiration for Maverick's role and asks about Iceman. Maverick mentions they fought with Iceman yesterday because they thought they had become friends but things went back to usual. The AI then warns Maverick that Iceman is coming."),
  HumanMessage(content='Same as usual, I thought that we became a friend but...'),
  AIMessage(content='Look, Iceman is comming!'),
  HumanMessage(content='Oh, shit. What should I do.'),
  AIMessage(content='Say something!')]}